In [1]:
import pandas as pd
import numpy as np

# Read in Data
data538 = pd.read_csv('nfl_elo.csv')
my_dict = {}

# Varying parameters
starting_epsilon = 1.0614
hfa_optimal = 57.0
epsilon_decay = 0.9863


def get_post_elo(home_team, home_team_elo, away_team, away_team_elo, point_diff, decay, home_adv):

    if point_diff > 0:
        FD = 1-get_win_prob(home_team_elo, away_team_elo, home_adv)
    else: FD = 0 - get_win_prob(home_team_elo, away_team_elo, home_adv)
    
    home_post_elo = get_points(point_diff, home_team_elo, away_team_elo, decay, home_adv)*FD  + home_team_elo
    away_post_elo = away_team_elo + (home_team_elo - home_post_elo)
    
    my_dict[home_team] = home_post_elo
    my_dict[away_team] = away_post_elo
    
    return home_post_elo, away_post_elo
    
def get_win_prob(home_team_elo, away_team_elo, hfa):
    home_team_elo += hfa
    return 1.0/(1.0+(10.0**((away_team_elo-home_team_elo)/400)))

def get_points(point_diff, team_elo, opp_elo, decay, HFA):
    
    if point_diff > 0:
        elo_dif = team_elo-opp_elo + HFA
    else: elo_dif = - (team_elo-opp_elo + HFA)
    
    K = 20.0 * np.log(np.abs(point_diff)+1) * 2.2/(elo_dif*0.001+2.2)
    
    return K*decay

games_2019 = data538[data538['season']==2019]

# Divide 2019 Games Into 'Played' and 'Not Played'
played = games_2019[~np.isnan(games_2019.score1)]
not_played = games_2019[np.isnan(games_2019.score1)] 



for i in range(played.shape[0]):

    # Get Game Info
    row = played.iloc[i]
    team = row.team1
    opp = row.team2
    point_diff = row.score1 - row.score2
    
    team_pre = row.elo1_pre
    opp_pre = row.elo2_pre
    
    # Adjust Epsilon Every 25 Games
    if i % 25 == 0:
        starting_epsilon *= epsilon_decay
    
    g = get_post_elo(team, team_pre, opp, opp_pre, point_diff, starting_epsilon, hfa_optimal)

    
for j in range(not_played.shape[0]):

    # Get Game Info
    row = not_played.iloc[j]
    team = row.team1
    opp = row.team2
    
    team_pre = my_dict[team]
    opp_pre = my_dict[opp]
    
    var = get_win_prob(team_pre, opp_pre, hfa_optimal)
    
    print(team)
    print(var)    

    g = get_post_elo(team, team_pre, opp, opp_pre, point_diff, starting_epsilon, hfa_optimal)

HOU
0.378242887843
SEA
0.573124225519
CHI
0.586467059895
CLE
0.777879503549
BUF
0.3607662692
HOU
0.793517901048
ATL
0.59843041246
NO
0.628763618242
NYJ
0.597598212724
TB
0.547293522863
GB
0.823635838843
MIN
0.809182888368
JAX
0.436512745846
NE
0.655009911365
ARI
0.273904554654
OAK
0.385135759247
LAR
0.470648506715
PHI
0.817763197214
BAL
0.86969793001
WSH
0.332289920311
KC
0.788140401681
CAR
0.307963915787
DET
0.402823579101
GB
0.588169514579
TEN
0.420487259346
CIN
0.12713684236
NYG
0.528409258997
PIT
0.459720501585
OAK
0.624959254181
ARI
0.441864905976
SF
0.729752677805
DAL
0.443507525926
LAC
0.357384596709
NO
0.807021350519
TB
0.429032723438
NE
0.665326493741
SF
0.639229057709
WSH
0.665882410988
MIA
0.483011859836
ATL
0.662744175077
IND
0.593828293928
NYJ
0.351913282711
TEN
0.387439199922
CLE
0.277464891524
DEN
0.547197220702
LAC
0.601484971905
PHI
0.473438127541
SEA
0.753285840462
CHI
0.485727607117
MIN
0.536427420198
HOU
0.51585494624
BUF
0.72366693136
NYG
0.300501426216
JAX
0.41306